In [ ]:
!pip install face_recognition
!pip install --upgrade pip
from PIL import Image
import cv2
import numpy as np
import face_recognition
import os
import shutil
from sklearn.metrics import explained_variance_score

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
from google.cloud import storage
import os

os.environ["Bangkit Capstone Project"] = "rock-web-312913"

GOOGLE_APPLICATION_CREDENTIALS='creds.json'
PROJECT_ID='rock-web-312913'

In [ ]:
#informasi path di gcloud dan local

#bucket cloud storage
train ='training_pictures-1'
test = 'test_pictures-1'

#local directory
local_train_dir_path = '/content/train'
local_test_dir_path = '/content/test'


In [ ]:
# Fungsi untuk mengecek list yang ada pada bucket test di cloud storage source blob
# dan membuat list yang ada di dalamnya

source_blob_test = []

def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"
    
    storage_client = storage.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    for blob in blobs:
        print(blob.name)
        source_blob_test.append(blob.name)

list_blobs(test)
print(source_blob_test)

In [ ]:
user = source_blob_test[0]

# Fungsi untuk mengganti nama blob yang ada pada bucket test di cloud storage

def rename_blob(bucket_name, blob_name, new_name):
    """Renames a blob."""
    # bucket_name = "your-bucket-name"
    # blob_name = "your-object-name"
    # new_name = "new-object-name"

    storage_client = storage.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    new_blob = bucket.rename_blob(blob, new_name)

    print("Blob {} has been renamed to {}".format(blob.name, new_blob.name))

rename_blob(test, user, user.lower())


In [ ]:
# Nama pengguna setelah di rename menjadi lower

username = user.lower()

print(username)

In [ ]:
# Fungsi untuk mengecek list yang ada pada bucket train di cloud storage source blob
# dan membuat list yang ada di dalamnya

source_blob_train = []

def list_blobs_train(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"
    
    storage_client = storage.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    for blob in blobs:
        print(blob.name)
        source_blob_train.append(blob.name)

list_blobs_train(train)


In [ ]:
#Download blob dari bucket untuk test

list_username_test = os.listdir(local_test_dir_path)

def download_blob_test(bucket_name, source_blob_name, destination_file_name):
    #bikin file kosong format jpg
    
    if username not in list_username_test:
      open(os.path.join("/content/", username), "w+")
      print("data berhasil dimasukkan")
      
      """Downloads a blob from the bucket."""
      # bucket_name = "your-bucket-name"
      # source_blob_name = "storage-object-name"
      # destination_file_name = "local/path/to/file"

      storage_client = storage.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
      bucket = storage_client.bucket(bucket_name)

      # Construct a client side representation of a blob.
      # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
      # any content from Google Cloud Storage. As we don't need additional data,
      # using `Bucket.blob` is preferred here.
      blob = bucket.blob(source_blob_name)
      blob.download_to_filename(destination_file_name)

      print(
          "Blob {} downloaded to {}.".format(
              source_blob_name, destination_file_name
          )
      )
      shutil.move("/content/{}".format(username), "/content/test/{}".format(username))
      print("Username berhasil dipindah ke directory test temporary")
    elif username in list_username:
      print("data gagal dimasukkan")

download_blob_test(test, username, username)

In [ ]:
#Download blob bucket untuk train

list_username_train = os.listdir(local_train_dir_path)

def download_blob_train(bucket_name, source_blob_name, destination_file_name):
    #bikin file kosong format jpg

    if username not in list_username:
      open(os.path.join("/content/", username), "w+")
      print("data berhasil dimasukkan")
      
      """Downloads a blob from the bucket."""
      # bucket_name = "your-bucket-name"
      # source_blob_name = "storage-object-name"
      # destination_file_name = "local/path/to/file"

      storage_client = storage.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
      bucket = storage_client.bucket(bucket_name)

      # Construct a client side representation of a blob.
      # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
      # any content from Google Cloud Storage. As we don't need additional data,
      # using `Bucket.blob` is preferred here.
      blob = bucket.blob(source_blob_name)
      blob.download_to_filename(destination_file_name)

      print(
          "Blob {} downloaded to {}.".format(
              source_blob_name, destination_file_name
          )
      )
      shutil.move("/content/{}".format(username), "/content/train/{}".format(username))
      print("Username berhasil dipindah ke directory train temporary")
    elif username in list_username:
      print("Data gagal dimasukkan")

if username in source_blob_train:
  download_blob_train(train, username, username)
else:
  print("Username tidak terdaftar")

In [ ]:
images = []
imagestest = []
className = []
myList = os.listdir(local_train_dir_path)
listTest = os.listdir(local_test_dir_path)
print(myList)
print(listTest)

In [ ]:
# Fungsi untuk meng-upload file test yang belum terdaftar ke bucket train di cloud storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} berhasil di upload ke {}.".format(
            source_file_name, destination_blob_name
        )
    )

In [ ]:
# fungsi untuk men-delete file test yang ada pada bucket test di cloud storage
# karena nantinya file yang tidak terdaftar akan dipindah ke bucket train di cloud storage

def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    # bucket_name = "your-bucket-name"
    # blob_name = "your-object-name"

    storage_client = storage.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.delete()

    print("Blob {} di bucket test pada cloud storage berhasil dihapus.".format(blob_name))

# Bagian Baru

In [ ]:
# Pattern matching antara train dan test pada local yang telah di download dari bucket

foto_test = (source_blob_test[0])
print(foto_test)

if foto_test in source_blob_train :
  for name in myList:
    curImage = cv2.imread(username, 0)
    images.append(curImage)  
    className.append(os.path.splitext(name)[0])
  print(className)
  imagestest = cv2.imread(username,0)

# pattern matching foto test dengan train

  train_image = face_recognition.load_image_file("/content/train/{}".format(str(username)))
  test_image = face_recognition.load_image_file("/content/test/{}".format(str(username)))

  train_image_encoding = face_recognition.face_encodings(train_image)[0]
  test_image_encoding = face_recognition.face_encodings(test_image)[0]

  results = face_recognition.compare_faces([train_image_encoding], test_image_encoding)

  print(results)

  acc = explained_variance_score((train_image_encoding), (test_image_encoding))
  print(acc)

# hapus file test
  os.remove("/content/test/{}".format(str(username)))

# hapus file train
  os.remove("/content/train/{}".format(str(username)))

# hapus file test yang ada di bucket test pada cloud storage
  delete_blob(test, username)

else:
  print("Data akan disimpan pada cloud storage")
  upload_blob(train, "/content/test/{}".format(str(username)), username)

# hapus file test
  os.remove("/content/test/{}".format(str(username)))

# hapus file test yang ada di bucket test pada cloud storage
  delete_blob(test, username)


